<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#数据库与集合（未完待续。。。）" data-toc-modified-id="数据库与集合（未完待续。。。）-1">数据库与集合（未完待续。。。）</a></span><ul class="toc-item"><li><span><a href="#集合的显示创建" data-toc-modified-id="集合的显示创建-1.1">集合的显示创建</a></span><ul class="toc-item"><li><span><a href="#例子" data-toc-modified-id="例子-1.1.1">例子</a></span><ul class="toc-item"><li><span><a href="#创建capped集合" data-toc-modified-id="创建capped集合-1.1.1.1">创建capped集合</a></span></li><li><span><a href="#创建一个具有文档验证的集合，mongo-shell" data-toc-modified-id="创建一个具有文档验证的集合，mongo-shell-1.1.1.2">创建一个具有文档验证的集合，mongo shell</a></span></li><li><span><a href="#pymongo操作" data-toc-modified-id="pymongo操作-1.1.1.3">pymongo操作</a></span></li><li><span><a href="#指定排序规则（collation）" data-toc-modified-id="指定排序规则（collation）-1.1.1.4">指定排序规则（collation）</a></span></li><li><span><a href="#[指定存储引擎]" data-toc-modified-id="[指定存储引擎]-1.1.1.5">[指定存储引擎]</a></span></li></ul></li></ul></li><li><span><a href="#文档验证" data-toc-modified-id="文档验证-1.2">文档验证</a></span></li><li><span><a href="#uuid" data-toc-modified-id="uuid-1.3">uuid</a></span></li></ul></li></ul></div>

# 数据库与集合（未完待续。。。）

MongoDB将数据记录存储为文档（BSON文档），这些文档一起收集在集合中。一个数据库存储一个或多个由文档组成的集合。

```
use myDB  # 选择一个数据库
show collections  # 显示该数据库下的所有集合
db.getCollectionInfos() # 查看集合详情
```

## 集合的显示创建

使用[db.createCollection()]显示创建，可设置集合的大小上限、文档验证规则等设置。

### 例子

#### 创建capped集合
log为名字，size是大小（bytes），max是最大的文档数
```
db.createCollection("log",{ capped:true, size:5242880m，max:5000})```

#### 创建一个具有文档验证的集合，mongo shell
具有文档验证的集合将每个插入或更新的文档与[validator]选项中指定的条件进行比较。如果文档不符合指定条件，根据validationLevel和validationAction，MongoDB要么返回警告，要么拒绝插入或更新文档。
```
db.createCollection(
"contacts",
    {
    validator:
        {
        $jsonSchema:
            {
            bsonType: "object",
            required: ["phone"],
            properties:
                {
                phone:
                    {
                    bsonType: "string",
                    description: "must be a string and is required"
                    },
                email:
                    {
                    bsonType: "string",
                    pattern: "@mongodb\.com$",
                    description: "must be a string and match the regular expression pattern"
                    },
                status: 
                    {
                    enum: ["unknown", "Incomplete"],
                    description: "can only be one of the enum values"
                    }
                }           
            }
        },
    validationAction: "warn"
    }
)
```

上面创建的集合要想插入文档，文档需满足必须：
- 含有phone字段，且value必须为string
- email有时必须满足xxx@mongodb.com的格式

更多的[Validation]查看

#### pymongo操作

In [31]:
import pymongo
import pprint
client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
db = client.test
js = {'bsonType': 'object',
      'required': ['phone'],
      'properties':{'phone':{'bsonType': 'string', 'description': 'must be a string and is required'}}}
val = {"$jsonSchema": js}
pprint.pprint(val)
db.create_collection('python1', validator=val)

{'$jsonSchema': {'bsonType': 'object',
                 'properties': {'phone': {'bsonType': 'string',
                                          'description': 'must be a string and '
                                                         'is required'}},
                 'required': ['phone']}}


Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'python1')

#### 指定排序规则（collation）
[collation]允许用户指定一些比较规则，例如大小写(lettercase)和声调符号(accent marks)

#### [指定存储引擎]

## 文档验证
默认情况下，集合中的文档不必保证具有相同的模式（schema）

但可以指定[文档验证规则]来强制约束文档的更新和插入操作

详情查看[模式验证]

## uuid
db.getCollectionInfos() 查看